In [1]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

!unzip '/content/drive/MyDrive/Colab Files/Car_YOLO.zip'

流式输出内容被截断，只能显示最后 5000 行内容。
  inflating: Car/images/Train/Gages Lake at North Mill East-7.jpg  
  inflating: Car/images/Train/Gages Lake at North Mill East-9.jpg  
  inflating: Car/images/Train/Gages Lake at North Mill West-0.jpg  
  inflating: Car/images/Train/Gages Lake at North Mill West-1.jpg  
  inflating: Car/images/Train/Gages Lake at North Mill West-2.jpg  
  inflating: Car/images/Train/Gages Lake at North Mill West-3.jpg  
  inflating: Car/images/Train/Gages Lake at North Mill West-4.jpg  
  inflating: Car/images/Train/Gages Lake at North Mill West-5.jpg  
  inflating: Car/images/Train/Gages Lake at North Mill West-6.jpg  
  inflating: Car/images/Train/Gages Lake at North Mill West-7.jpg  
  inflating: Car/images/Train/Gages Lake at North Mill West-9.jpg  
  inflating: Car/images/Train/Grand at McAree East-1.jpg  
  inflating: Car/images/Train/Grand at McAree East-2.jpg  
  inflating: Car/images/Train/Grand at McAree East-4.jpg  
  inflating: Car/images/Train/Grand at McAree Ea

In [2]:
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 644.5/644.5 kB 11.1 MB/s eta 0:00:00


In [3]:
import os
from ultralytics import YOLO

In [4]:
# Create YAML file
dataset_path = "/content/Car"
data_yaml = f"""
train: {dataset_path}/images/Train/
val: {dataset_path}/images/Val/
test: {dataset_path}/images/Test/
nc: 5
names: ['Truck','Sedan','SUV','Motorbike','Bus']
"""
with open("/content/Car/data.yaml", "w") as f:
    f.write(data_yaml)

In [5]:
train_folder = '/content/Car/images/Train/'
val_folder = '/content/Car/images/Val/'
test_folder = '/content/Car/images/Test/'

In [6]:
def create_txt_file(folder_path, txt_path):
    image_files = sorted([f for f in os.listdir(folder_path) if f.endswith('.jpg')])
    with open(txt_path, 'w') as file:
        for img_file in image_files:
            file.write(os.path.join(folder_path, img_file) + '\n')

In [7]:
create_txt_file(train_folder, '/content/Car/train.txt')
create_txt_file(val_folder, '/content/Car/val.txt')
create_txt_file(test_folder, '/content/Car/test.txt')

In [8]:
model = YOLO('yolov8m.yaml').load('yolov8m.pt')


                   from  n    params  module                                       arguments                     
  0                  -1  1      1392  ultralytics.nn.modules.conv.Conv             [3, 48, 3, 2]                 
  1                  -1  1     41664  ultralytics.nn.modules.conv.Conv             [48, 96, 3, 2]                
  2                  -1  2    111360  ultralytics.nn.modules.block.C2f             [96, 96, 2, True]             
  3                  -1  1    166272  ultralytics.nn.modules.conv.Conv             [96, 192, 3, 2]               
  4                  -1  4    813312  ultralytics.nn.modules.block.C2f             [192, 192, 4, True]           
  5                  -1  1    664320  ultralytics.nn.modules.conv.Conv             [192, 384, 3, 2]              
  6                  -1  4   3248640  ultralytics.nn.modules.block.C2f             [384, 384, 4, True]           
  7                  -1  1   1991808  ultralytics.nn.modules.conv.Conv             [384

In [9]:
results = model.train(data='/content/Car/data.yaml', epochs=100, imgsz=640)

Ultralytics YOLOv8.0.199 🚀 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8m.yaml, data=/content/Car/data.yaml, epochs=100, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, stream_buffer=False, line_width=None, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, boxes=True, format=torchscript, keras=False, optim

In [10]:
model = YOLO('/content/runs/detect/train/weights/best.pt')
model.export()

Ultralytics YOLOv8.0.199 🚀 Python-3.10.12 torch-2.0.1+cu118 CPU (Intel Xeon 2.20GHz)
YOLOv8m summary (fused): 218 layers, 25842655 parameters, 0 gradients, 78.7 GFLOPs

PyTorch: starting from '/content/runs/detect/train/weights/best.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 9, 8400) (49.6 MB)

TorchScript: starting export with torch 2.0.1+cu118...
TorchScript: export success ✅ 3.4s, saved as '/content/runs/detect/train/weights/best.torchscript' (99.1 MB)

Export complete (6.2s)
Results saved to /content/runs/detect/train/weights
Predict:         yolo predict task=detect model=/content/runs/detect/train/weights/best.torchscript imgsz=640  
Validate:        yolo val task=detect model=/content/runs/detect/train/weights/best.torchscript imgsz=640 data=/content/Car/data.yaml  
Visualize:       https://netron.app


'/content/runs/detect/train/weights/best.torchscript'

In [11]:
model = YOLO('/content/runs/detect/train/weights/best.pt')
model.export(format='engine')

WARNING ⚠️ TensorRT requires GPU export, automatically assigning device=0
Ultralytics YOLOv8.0.199 🚀 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)
YOLOv8m summary (fused): 218 layers, 25842655 parameters, 0 gradients, 78.7 GFLOPs

PyTorch: starting from '/content/runs/detect/train/weights/best.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 9, 8400) (49.6 MB)
requirements: Ultralytics requirement ['nvidia-tensorrt'] not found, attempting AutoUpdate...
Looking in indexes: https://pypi.ngc.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 340.9/340.9 MB 310.7 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 333.1/333.1 MB 277.7 MB/s 

============= Diagnostic Run torch.onnx.export version 2.0.1+cu118 =============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================



ONNX: export success ✅ 18.4s, saved as '/content/runs/detect/train/weights/best.onnx' (98.8 MB)

TensorRT: starting export with TensorRT 8.4.3.1...
TensorRT: input "images" with shape(1, 3, 640, 640) DataType.FLOAT
TensorRT: output "output0" with shape(1, 9, 8400) DataType.FLOAT
TensorRT: building FP32 engine as /content/runs/detect/train/weights/best.engine
TensorRT: export success ✅ 133.1s, saved as '/content/runs/detect/train/weights/best.engine' (110.9 MB)

Export complete (133.5s)
Results saved to /content/runs/detect/train/weights
Predict:         yolo predict task=detect model=/content/runs/detect/train/weights/best.engine imgsz=640  
Validate:        yolo val task=detect model=/content/runs/detect/train/weights/best.engine imgsz=640 data=/content/Car/data.yaml  
Visualize:       https://netron.app


'/content/runs/detect/train/weights/best.engine'

In [12]:
model = YOLO('/content/runs/detect/train/weights/best.pt')
model.export(format='tflite')
model.export(format='edgetpu')

Ultralytics YOLOv8.0.199 🚀 Python-3.10.12 torch-2.0.1+cu118 CPU (Intel Xeon 2.20GHz)
YOLOv8m summary (fused): 218 layers, 25842655 parameters, 0 gradients, 78.7 GFLOPs

PyTorch: starting from '/content/runs/detect/train/weights/best.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 9, 8400) (49.6 MB)
requirements: Ultralytics requirements ['onnx2tf>=1.15.4,<=1.17.5', 'sng4onnx>=1.0.1', 'onnx_graphsurgeon>=0.3.26', 'tflite_support'] not found, attempting AutoUpdate...
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.4/400.4 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.1/42.1 kB 242.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 MB 28.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.7/227.7 kB 272.0 MB/s eta 0:00:00

requirements: AutoUpdate success ✅ 14.3s, installed 4 packages: ['onnx2tf>=1.15.4,<=1.17.5', 'sng4onnx>=1.0.

============= Diagnostic Run torch.onnx.export version 2.0.1+cu118 =============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================



ONNX: export success ✅ 15.6s, saved as '/content/runs/detect/train/weights/best.onnx' (98.8 MB)
TensorFlow SavedModel: running 'onnx2tf -i "/content/runs/detect/train/weights/best.onnx" -o "/content/runs/detect/train/weights/best_saved_model" -nuo --non_verbose'
TensorFlow SavedModel: export success ✅ 51.9s, saved as '/content/runs/detect/train/weights/best_saved_model' (247.1 MB)

TensorFlow Lite: starting export with tensorflow 2.13.0...
TensorFlow Lite: export success ✅ 0.0s, saved as '/content/runs/detect/train/weights/best_saved_model/best_float32.tflite' (98.8 MB)

Export complete (54.2s)
Results saved to /content/runs/detect/train/weights
Predict:         yolo predict task=detect model=/content/runs/detect/train/weights/best_saved_model/best_float32.tflite imgsz=640  
Validate:        yolo val task=detect model=/content/runs/detect/train/weights/best_saved_model/best_float32.tflite imgsz=640 data=/content/Car/data.yaml  
Visualize:       https://netron.app
Ultralytics YOLOv8.0.1

============= Diagnostic Run torch.onnx.export version 2.0.1+cu118 =============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================



ONNX: export success ✅ 15.5s, saved as '/content/runs/detect/train/weights/best.onnx' (98.8 MB)
TensorFlow SavedModel: running 'onnx2tf -i "/content/runs/detect/train/weights/best.onnx" -o "/content/runs/detect/train/weights/best_saved_model" -nuo --verbosity info -oiqt -qt per-tensor'
TensorFlow SavedModel: export success ✅ 562.2s, saved as '/content/runs/detect/train/weights/best_saved_model' (322.1 MB)
Edge TPU: WARNING ⚠️ Edge TPU known bug https://github.com/ultralytics/ultralytics/issues/1185

Edge TPU: export requires Edge TPU compiler. Attempting install from https://coral.ai/docs/edgetpu/compiler/

Edge TPU: starting export with Edge TPU compiler 16.0.384591198...
Edge TPU: running 'edgetpu_compiler -s -d -k 10 --out_dir "/content/runs/detect/train/weights/best_saved_model" "/content/runs/detect/train/weights/best_saved_model/best_full_integer_quant.tflite"'
Edge TPU: export success ✅ 110.1s, saved as '/content/runs/detect/train/weights/best_saved_model/best_full_integer_quant

'/content/runs/detect/train/weights/best_saved_model/best_full_integer_quant_edgetpu.tflite'

In [13]:
# Create YAML file
dataset_path = "/content/Car"
data_yaml = f"""
train: {dataset_path}/images/Train/
val: {dataset_path}/images/Test/
test: {dataset_path}/images/Val/
nc: 5
names: ['Truck','Sedan','SUV','Motorbike','Bus']
"""
with open("/content/Car/data.yaml", "w") as f:
    f.write(data_yaml)

model.val(data='/content/Car/data.yaml')  # Validate the model on the data you trained on

Ultralytics YOLOv8.0.199 🚀 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)
YOLOv8m summary (fused): 218 layers, 25842655 parameters, 0 gradients, 78.7 GFLOPs
val: Scanning /content/Car/labels/Test... 292 images, 3 backgrounds, 0 corrupt: 100%|██████████| 295/295 [00:00<00:00, 1382.48it/s]
val: New cache created: /content/Car/labels/Test.cache
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):  11%|█         | 2/19 [00:03<00:34,  2.00s/it]WARNING ⚠️ NMS time limit 1.300s exceeded
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 19/19 [00:12<00:00,  1.51it/s]
                   all        295       2468      0.686      0.618      0.638      0.488
                 Truck        295        205      0.625      0.595      0.649      0.489
                 Sedan        295        929      0.464      0.864       0.71      0.549
                   SUV        295       1304      0.792    

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3, 4])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x78d704282fe0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
        